In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
#ds = load_dataset('json', data_files='/content/drive/MyDrive/IMDB_reviews.json')

In [ ]:
#ds

DatasetDict({
    train: Dataset({
        features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary'],
        num_rows: 573913
    })
})

In [ ]:
# loading reviews dataset

import json
import pandas as pd

ds_reviews = pd.read_json('/content/drive/MyDrive/IMDB_reviews.json', lines=True)
ds_reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [ ]:
# creating a smaller dataset of 30000 examples for train and validation
# balanced number of spoiler and no spoiler reviews

spoiler_reviews = ds_reviews.loc[ds_reviews['is_spoiler'] == True]

small_spoiler_train = spoiler_reviews.sample(n=15000)

no_spoiler_reviews = ds_reviews.loc[ds_reviews['is_spoiler'] == False]

small_no_spoiler_train = no_spoiler_reviews.sample(n=15000)

train_reviews = pd.concat([small_spoiler_train, small_no_spoiler_train], ignore_index = False)
train_reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
479746,8 May 2006,tt0099674,ur1887253,True,"I had always heard that part III sucked, so I ...",10,I wish I had known what a great movie this was...
554203,2 April 2014,tt0122690,ur26154754,True,Ronin is an example of a film that had great p...,5,In a word; 'Frustrating'
187203,3 June 2017,tt1300854,ur25067338,True,"To this day, I'm still puzzled as to why they ...",5,"For an Iron Man film, it's fairly mediocre."
153735,11 April 2017,tt1716772,ur68331092,True,​Plot Overview: When Simon(Joe Thomas)'s girlf...,8,Classic Lads Holiday. Would Recommend.
509257,14 January 2004,tt0110632,ur2202602,True,Natural Born Killers is a truly tumultuous rid...,6,"Tries hard, but never accomplishes anything"
...,...,...,...,...,...,...,...
360720,11 April 2004,tt0365748,ur1436086,False,Absolutely first rate.It lives up to all the p...,10,As good as it gets.
287665,13 February 2001,tt0190332,ur0444309,False,"The special effects were amazing, and the batt...",10,By far the best Foreign Language film I have e...
194526,22 June 2013,tt0816711,ur37812556,False,I would to start my comment with 2 things: Whe...,7,"Not as bad as they said, but not good as we ho..."
210203,24 March 2015,tt1528100,ur28749720,False,"""Exodus"" is like a bag of mixed nuts, just as ...",4,thou shalt not watch!


In [ ]:
# from pandas to Dataset

from datasets import Dataset
reviews_ds = Dataset.from_pandas(train_reviews)
reviews_ds

Dataset({
    features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__'],
    num_rows: 30000
})

In [ ]:
# creating a smaller dataset of 6000 examples for evaluation (testing)
# balanced number of spoiler and no spoiler reviews

small_spoiler_test = spoiler_reviews.sample(n=3000)
small_no_spoiler_test = no_spoiler_reviews.sample(n=3000)

test_reviews = pd.concat([small_spoiler_test, small_no_spoiler_test], ignore_index = False)
test_reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
149112,8 August 2011,tt0945513,ur23754677,True,"This film begins good, fast and exciting. Howe...",2,Writers kept forgetting the point of the movie...
137231,10 July 2010,tt1424381,ur3830691,True,There's only a minor spoiler at the bottom. I ...,10,Perfect!
135753,24 February 2011,tt0964517,ur25140755,True,"How many boxing movies do we need? Apparently,...",8,"How many boxing movies do we need? Apparently,..."
471591,13 December 2009,tt0780521,ur20400868,True,I liked this movies as much as I liked Aladdin...,9,one of the best Disney movies since the 90's
203,14 June 2017,tt0111161,ur77355001,True,This movie is by far one of the best movies I ...,10,Review
...,...,...,...,...,...,...,...
281026,7 December 2003,tt0249462,ur0835751,False,Various aspects of life are shown in this exce...,8,Life teaches all of us
104788,15 February 2002,tt0097165,ur1602286,False,Dead Poets Society is the best movie I have ev...,10,Praise for the film Dead Poets Society
87512,4 December 2014,tt0052618,ur26835138,False,"The same quality that made epics like ""Star Wa...",9,Charlton Hestons Best Film
396621,29 March 2007,tt0416449,ur14793718,False,That Was Ridiculous ... Nothing More Nothing L...,1,Damn


In [ ]:
small_ds_eval_0 = Dataset.from_pandas(test_reviews)
small_ds_eval_0

Dataset({
    features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__'],
    num_rows: 6000
})

In [ ]:
# loading movie details dataset (where there is film plot summary)

ds_info = load_dataset('json', data_files='/content/drive/MyDrive/IMDB_movie_details.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-90be2aff2566985f/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
ds_info

DatasetDict({
    train: Dataset({
        features: ['movie_id', 'plot_summary', 'duration', 'genre', 'rating', 'release_date', 'plot_synopsis'],
        num_rows: 1572
    })
})

In [ ]:
ds_info['train'][:2]

{'movie_id': ['tt0105112', 'tt1204975'],
 'plot_summary': ["Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they tr

In [ ]:
# dict of movie ids and summaries

summaries = {}
#synopsises = {}

for i in range(len(ds_info['train']['movie_id'])):

    id = ds_info['train']['movie_id'][i]

    su = ds_info['train']['plot_summary'][i]
    summaries[id] = su

    #sy = ds_info['train']['plot_synopsis'][i]
    #synopsises[id] = sy

In [ ]:
print(summaries)

{'tt0105112': "Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they try to find the man before he makes another att

In [ ]:
# adding plot to features in dataset function

def add_plot(examples):

    mov_id = examples['movie_id']
    if mov_id in summaries:
        examples['plot_summary'] = summaries[mov_id]
    else:
        examples['plot_summary'] = ''

    return examples

In [ ]:
# train/val split

from sklearn.model_selection import train_test_split
small_ds_0 = reviews_ds.train_test_split(test_size=0.2, seed=42)

In [ ]:
small_ds_0

DatasetDict({
    train: Dataset({
        features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__'],
        num_rows: 6000
    })
})

In [ ]:
small_ds_0['train']

Dataset({
    features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__'],
    num_rows: 24000
})

In [ ]:
# ?
#small_ds_0 = ds_reviews.filter(lambda example, index: index % 20 == 0, with_indices=True) #class_weight='balanced',

Filter:   0%|          | 0/459130 [00:00<?, ? examples/s]

Filter:   0%|          | 0/114783 [00:00<?, ? examples/s]

In [ ]:
small_ds = small_ds_0.map(add_plot)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
small_ds_eval = small_ds_eval_0.map(add_plot)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
s = [i for i in small_ds['train']['is_spoiler'] if i == True]
print(len(s))

12024


In [ ]:
s = [i for i in small_ds['test']['is_spoiler'] if i == True]
print(len(s))

2976


In [ ]:
small_ds

DatasetDict({
    train: Dataset({
        features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__', 'plot_summary'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text', 'rating', 'review_summary', '__index_level_0__', 'plot_summary'],
        num_rows: 6000
    })
})

In [ ]:
################################################################

In [ ]:
import json
import pandas as pd
#import read_json

with open('/content/drive/MyDrive/IMDB_reviews.json') as json_file:
    d = json.load(json_file)

#ds_reviews = json.loads('/content/drive/MyDrive/IMDB_reviews.json')

In [ ]:
#####################################################

In [ ]:
RANDOM_SEED = 4

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=ae658efd73264404481b9b05ad2936b454afbc27af64da59d434baf8a9736887
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score


In [ ]:
from datasets import load_dataset, load_metric

rouge_metric = load_metric("rouge")
meteor_metric = load_metric("meteor")

<ipython-input-29-e759cface4f8>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "text2text-generation: "
else:
    prefix = ''

In [ ]:
def bool_to_text(example):
    if example['is_spoiler'] == True:
        example['is_spoiler'] = 'yes'
    else:
        example['is_spoiler'] = 'no'
    return example

In [ ]:
# without plot summary

In [ ]:
ds_rev_0 = small_ds_0.map(bool_to_text)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
ds_rev_0['train'][0]

{'review_date': '12 January 2000',
 'movie_id': 'tt0119094',
 'user_id': 'ur0549041',
 'is_spoiler': 'no',
 'review_text': 'This is a John Woo masterpiece. After he missed with "Hard Target" and "Broken Arrow," he took a 360 degree turn into the good stuff. John Travolta and Nicholas Cage give excellent performances. The action was filmed with great excitement. There were explosions, gunfire, more explosions, and more gunfire. A must-see for all John Woo/action fans.',
 'rating': 9,
 'review_summary': 'The best action movie ever made',
 '__index_level_0__': 537279}

In [ ]:
eval_0 = small_ds_eval_0.map(bool_to_text)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
max_input_length = 512
max_target_length = 10

def preprocess_function(examples):
    inf = []
    #print(examples)
    for i in range(len(examples['review_text'])):
        #print(examples['context'][i])
        review = ''
        review += 'review: ' + examples['review_text'][i]
        inf.append(review)
    inputs = [prefix + doc for doc in inf]
    #inputs = [prefix + doc for doc in examples['review_text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    #answ = []
    #for j in range(len(examples['is_spoiler'])):
        #answ.append(examples['is_spoiler'][j])
    #print(type(answ[0]))
    labels = tokenizer(text_target=examples['is_spoiler'], max_length=max_target_length, truncation=True)
    #print(tokenizer.convert_ids_to_tokens(labels))


    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
tokenized_dataset = ds_rev_0.map(preprocess_function, batched=True)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = eval_0.map(preprocess_function, batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
tokenized = preprocess_function(ds_rev_0['train'][:2])
tokenized

{'input_ids': [[1499, 357, 6327, 18, 10877, 10, 1132, 10, 6902, 6, 3, 99, 27, 1114, 12, 217, 430, 5640, 13, 37, 15090, 7, 13, 1626, 5271, 986, 6, 3, 23, 133, 43, 131, 2120, 30, 8, 332, 5, 553, 5, 11, 7533, 46, 625, 5640, 13, 34, 5, 100, 410, 59, 1727, 114, 3, 9, 1974, 8, 1297, 97, 24, 27, 7533, 34, 5, 21309, 6922, 57, 18398, 25649, 1420, 6, 1119, 12, 1810, 114, 3, 9, 684, 4940, 6, 405, 59, 161, 5, 37, 880, 13, 8, 4061, 47, 5676, 38, 168, 5, 16908, 21965, 341, 19, 1312, 38, 664, 6, 68, 8, 163, 589, 24, 2697, 140, 1638, 47, 8, 3, 7990, 858, 3112, 45, 2011, 7080, 32, 4660, 5, 27, 317, 24, 47, 8, 80, 97, 27, 24524, 5, 37, 1974, 47, 59, 6613, 6, 737, 31, 17, 143, 1254, 6, 11, 47, 17509, 6640, 5, 27, 278, 31, 17, 317, 48, 1974, 225, 43, 237, 118, 1883, 5, 242, 273, 13, 25, 24, 410, 281, 12, 217, 34, 6, 27, 183, 8032, 25, 141, 12, 2561, 190, 34, 5, 3, 26909, 6, 213, 47, 8, 8, 733, 16, 48, 1974, 10769, 94, 47, 66, 147, 8, 286, 5, 100, 47, 1077, 8, 6025, 1974, 27, 43, 894, 437, 37, 901, 265, 32

In [ ]:
tokenizer.convert_ids_to_tokens(tokenized[0])

TypeError: ignored

In [ ]:
# 2 with plot summary

In [ ]:
ds_rev = small_ds.map(bool_to_text)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
eval = small_ds_eval.map(bool_to_text)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
max_input_length = 512
max_target_length = 10

def preprocess_function(examples):
    inf = []
    #print(examples)
    for i in range(len(examples['review_text'])):
        #print(examples['context'][i])
        review = ''
        review += 'review: ' + examples['review_text'][i]
        plot = ''
        plot += 'plot summary: ' + examples['plot_summary'][i]
        info = ''
        info += review + ' ' + plot
        inf.append(info)
    inputs = [prefix + doc for doc in inf]
    #inputs = [prefix + doc for doc in examples['review_text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    #answ = []
    #for j in range(len(examples['is_spoiler'])):
        #answ.append(examples['is_spoiler'][j])
    #print(type(answ[0]))
    labels = tokenizer(text_target=examples['is_spoiler'], max_length=max_target_length, truncation=True)
    #print(tokenizer.convert_ids_to_tokens(labels))


    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
tokenized_dataset = ds_rev.map(preprocess_function, batched=True)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = eval.map(preprocess_function, batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
# learning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.3 MB/s eta 0:00:00


In [ ]:
import nltk
import numpy as np

import numpy as np
import evaluate

clf_metrics = evaluate.combine(["accuracy", "precision", "recall"])

#def compute_metrics(eval_pred):
    #logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=-1)
    #return clf_metrics.compute(predictions=predictions, references=labels)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    #print(decoded_preds[:100])
    #print(decoded_labels[:100])

    class_preds = []
    class_labels = []

    for i in decoded_preds:
        if i == 'yes':
            class_preds.append(1)
        else:
            class_preds.append(0)

    for j in decoded_labels:
        if j == 'yes':
            class_labels.append(1)
        else:
            class_labels.append(0)
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    result.update(meteor_metric.compute(predictions=decoded_preds, references=decoded_labels))
    #return {k: round(v, 4) for k, v in result.items()}, clf_metrics.compute(predictions=class_preds, references=class_labels)
    return clf_metrics.compute(predictions=class_preds, references=class_labels)

In [ ]:
#

clf_metrics = evaluate.combine(["accuracy", "precision", "recall"])

def compute_metrics(eval_pred):
    logits, lab = eval_pred
    print(logits[:100])
    print(lab[:100])
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(logits, axis=-1)
    print(predictions[:100])
    print(labels[:100])
    return clf_metrics.compute(predictions=predictions, references=labels)

In [ ]:
# 1 without plot summary

In [ ]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    #f"{model_name}-finetuned-classification",
    output_dir="./t5-small-classify1", #The output directory
    overwrite_output_dir=True,
    #evaluation_strategy = "epoch", 
    evaluation_strategy="steps",
    eval_steps=500, #200
    save_strategy="steps",
    save_steps=500, #200
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, # 0.5
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall
500,0.342600,0.323965,0.633000,0.703684,0.449261
1000,0.335800,0.320872,0.647500,0.695415,0.514785
1500,0.338200,0.313841,0.651500,0.690323,0.539315


TrainOutput(global_step=1500, training_loss=0.3388524169921875, metrics={'train_runtime': 1122.0859, 'train_samples_per_second': 42.777, 'train_steps_per_second': 1.337, 'total_flos': 6496406470656000.0, 'train_loss': 0.3388524169921875, 'epoch': 2.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_test)

{'eval_loss': 0.3101401627063751,
 'eval_accuracy': 0.6573333333333333,
 'eval_precision': 0.7143505903723887,
 'eval_recall': 0.5243333333333333,
 'eval_runtime': 105.152,
 'eval_samples_per_second': 57.06,
 'eval_steps_per_second': 1.788,
 'epoch': 2.0}

In [ ]:
#

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,0.824800,0.284999,74.425100,0.000000,74.407700,74.442500,2.000000,0.372000
2,0.304100,0.279961,74.372800,0.000000,74.355400,74.390200,2.000000,0.371800


TrainOutput(global_step=1436, training_loss=0.48282134499722534, metrics={'train_runtime': 969.3551, 'train_samples_per_second': 47.366, 'train_steps_per_second': 1.481, 'total_flos': 6214083472785408.0, 'train_loss': 0.48282134499722534, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2799607217311859,
 'eval_rouge1': 74.3728,
 'eval_rouge2': 0.0,
 'eval_rougeL': 74.3554,
 'eval_rougeLsum': 74.3902,
 'eval_gen_len': 2.0,
 'eval_meteor': 0.3718,
 'eval_runtime': 102.5625,
 'eval_samples_per_second': 55.966,
 'eval_steps_per_second': 1.755,
 'epoch': 2.0}

In [ ]:
#

In [ ]:
trainer.save_model()

In [ ]:
# 2 with plot summary

In [ ]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    #f"{model_name}-finetuned-classification",
    output_dir="./t5-small-classify2", #The output directory
    overwrite_output_dir=True,
    #evaluation_strategy = "epoch", 
    evaluation_strategy="steps",
    eval_steps=500, #200
    save_strategy="steps",
    save_steps=500, #200
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2, # 0.5
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall
500,0.983300,0.337738,0.581333,0.655288,0.328965
1000,0.370600,0.340265,0.589333,0.690476,0.311828
1500,0.361200,0.337791,0.594333,0.696377,0.322917


TrainOutput(global_step=1500, training_loss=0.571697031656901, metrics={'train_runtime': 1125.3818, 'train_samples_per_second': 42.652, 'train_steps_per_second': 1.333, 'total_flos': 6496406470656000.0, 'train_loss': 0.571697031656901, 'epoch': 2.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_test)

{'eval_loss': 0.3380131721496582,
 'eval_accuracy': 0.582,
 'eval_precision': 0.673974540311174,
 'eval_recall': 0.31766666666666665,
 'eval_runtime': 104.8517,
 'eval_samples_per_second': 57.224,
 'eval_steps_per_second': 1.793,
 'epoch': 2.0}

In [ ]:
#

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Meteor
1,0.843800,0.295638,74.407700,0.000000,74.407700,74.425100,2.000000,0.372000


TrainOutput(global_step=718, training_loss=0.6856449414096503, metrics={'train_runtime': 489.0278, 'train_samples_per_second': 46.944, 'train_steps_per_second': 1.468, 'total_flos': 3107041736392704.0, 'train_loss': 0.6856449414096503, 'epoch': 1.0})

In [ ]:
trainer.save_model()

In [ ]:
# pipeline

In [ ]:
from transformers import pipeline

answerer = pipeline("text2text-generation", model = "./t5-small-classify2", tokenizer = AutoTokenizer.from_pretrained("t5-small"))

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
small_ds_eval[4000]

{'review_date': '6 February 2012',
 'movie_id': 'tt0366548',
 'user_id': 'ur31282285',
 'is_spoiler': False,
 'review_text': "I saw this movie with my little brother, and although the sequel is far worse, (probably deserving of a negative score) this movie did not live up to any expectations that we had. the humour and wit was stale, the characters were shallow and predictable, and the plot was the same old repetitive sequence you've seen time and time again in these animated flicks. This movie could have been a hundred times better with puppets, something Jim Henson style, but they always go for the cheap and cheesy animation these days, and the product of that is a seemingly endless stream of easily forgotten movies that are saturating the film industry and turning the video game industry into a bad scene of movie games as well. I disapprove of this kind of garbage, if I were you, I would avoid these cheap animated films at all costs, demand something more creative.",
 'rating': 1,
 

In [ ]:
prompt = ''
prefix = 'text2text-generation: '
review = 'review: '
review += small_ds_eval[2000]['review_text']
summary = 'plot_summary: '
summary += small_ds_eval[2000]['plot_summary']
prompt += prefix + ' ' + review + ' ' + summary
print(prompt)

text2text-generation:  review: Here's TRON: A Real cool guy driving around on his bike, living' on the edge cause "BOHOOO My father left me". He base-jumps from a skyscraper and...never mind.I wanted to walk out of the theater at least 14 times. And additionally one time when a ninja-program took a back flip and landed the COOL way, looking down, then slowly looking up. F(/&#! Endless over choreographed fighting-scenes, programs eating a pig for dinner...At that point my synapses overloaded and i felt like that guy in clockwork orange. I wondered if these programs would fart after eating that pork. In this perfected universe they have guns that jams. And why do they have big hangars for planes that actually fits inside a small stick anyway. The universe, the plot, the whole premise is built on a retarded thought you had the first time you smoked weed. Where is this actually happening? On the hard drive? Or maybe in the RAM, driving on light-bikes into the CPU...as a program..not litera

In [ ]:
answerer(prompt)

[{'generated_text': 'no'}]

In [ ]:
prompt = ''
prefix = 'text2text-generation: '
review = 'review: '
review += small_ds_eval[2000]['review_text']
summary = 'plot_summary: '
summary += small_ds_eval[2000]['plot_summary']
prompt += prefix + ' ' + review + ' ' + summary
print(prompt)

text2text-generation:  review: Here's TRON: A Real cool guy driving around on his bike, living' on the edge cause "BOHOOO My father left me". He base-jumps from a skyscraper and...never mind.I wanted to walk out of the theater at least 14 times. And additionally one time when a ninja-program took a back flip and landed the COOL way, looking down, then slowly looking up. F(/&#! Endless over choreographed fighting-scenes, programs eating a pig for dinner...At that point my synapses overloaded and i felt like that guy in clockwork orange. I wondered if these programs would fart after eating that pork. In this perfected universe they have guns that jams. And why do they have big hangars for planes that actually fits inside a small stick anyway. The universe, the plot, the whole premise is built on a retarded thought you had the first time you smoked weed. Where is this actually happening? On the hard drive? Or maybe in the RAM, driving on light-bikes into the CPU...as a program..not litera

In [ ]:
answerer(prompt)

[{'generated_text': 'no'}]